In [ ]:
# 이거 실행하고 다시 런타임 시작
# 한글 폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from pykrx import stock
sns.set(font='NaNumBarunGothic')

import tensorflow as tf


from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.svm import OneClassSVM
import matplotlib.pyplot as plt
%matplotlib inline

# Training and Predictions
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split

#------------------------------------------------------------------ CART(Classification and Regression Tree)
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVC, NuSVR
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor,  AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet  #----------------*****


#------------------------------------------------------------------ 증강학습
from sklearn.model_selection import KFold,cross_val_score,GridSearchCV
from imblearn.over_sampling import SMOTE

#------------------------------------------------------------------ 스케일링
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler


#------------------------------------------------------------------ 평가관련
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
# MAE
# MSE  : (squared=True)
# RMSE : (squared=False)
# MSLE

#------------------------------------------------------------------ 튜닝관련
# import optuna


from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import re
pd.options.display.max_columns = None

In [ ]:
baseline = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/real_final_data_for_modeling.csv')

In [ ]:
baseline

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,F_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,업종,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률
0,389500,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,2.0,1.0,2.0,0.0,1.0,0.000000,3.0,0.023564,1.0,14.593801,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,0,0,2018,110.399585,-72.542571,263.452137,0.047736,-5.930419
1,296640,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,1.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.006766,1.0,17.328700,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2018,90.942256,-63.629994,396.408093,0.470922,-0.354859
2,417970,23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,2.0,2.0,2.0,5.0,2.0,0.000000,3.0,0.000343,1.0,26.200702,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,57.427845,-3.584996,595.858592,0.863325,0.549708
3,314140,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,1.0,3.0,2.0,3.0,2.0,0.000000,3.0,0.000461,1.0,21.578064,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114
4,129920,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.001602,1.0,20.705556,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,950110,78.37,-3.08,-3.40,0.92,3.84,34.32,23.90,5.72,17.63,467.08,117.88,0.16,0.70,7.052749e+09,-8.972330e+10,1.0,0.0,1.0,4.0,1.0,0.965841,2.0,0.005466,1.0,15.903376,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2021,110.399585,-72.542571,263.452137,0.047736,-5.930419
5112,46970,2.98,99.94,-0.77,5.31,22.63,88.70,13.82,3.24,64.99,53.87,154.51,1.64,3.31,1.335246e+10,-2.713149e+09,1.0,2.0,2.0,4.0,1.0,3.323928,0.0,0.000051,1.0,15.822916,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,1.0,3,1,1,1,2021,4.120000,1.520000,464.000000,0.000000,-2.506494
5113,113810,-1.44,-4.48,-0.65,-1.02,0.04,0.16,0.05,-1.31,32.45,208.15,24.23,0.78,2.38,2.625700e+07,2.652719e+09,0.0,1.0,2.0,2.0,2.0,0.806495,2.0,0.002698,1.0,14.801748,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3,1,0,1,2021,7.880000,1.340000,115.000000,0.000000,-1.618280
5114,155650,4.66,7.88,22.56,10.32,8.13,89.91,5.03,6.38,80.72,23.88,897.93,1.62,2.03,8.979293e+09,6.462723e+09,2.0,3.0,2.0,3.0,2.0,7.915767,0.0,0.000030,1.0,31.194817,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2021,13.510000,2.130000,525.000000,0.000000,-0.354244


# F-score 점수 다시 넣기

In [ ]:
baseline['F_score'].value_counts()

1.0    3602
0.0     760
2.0     754
Name: F_score, dtype: int64

In [ ]:
f_score = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/부실지표/piotroski.csv')
f_score

,Stock,2018_F_score,2019_F_score,2020_F_score,2021_F_score
0,389500,4,3,5,6
1,296640,6,8,6,7
2,417970,7,7,7,8
3,314140,6,5,8,8
4,129920,5,6,4,7
...,...,...,...,...,...
1274,950110,4,5,7,4
1275,46970,7,5,6,6
1276,113810,5,7,5,6
1277,155650,7,2,8,7


In [ ]:
df_2018 = f_score[['Stock', '2018_F_score']]
df_2018.columns = ['Stock', 'F_score_raw']

df_2019 = f_score[['Stock', '2019_F_score']]
df_2019.columns = ['Stock', 'F_score_raw']

df_2020 = f_score[['Stock', '2020_F_score']]
df_2020.columns = ['Stock', 'F_score_raw']

df_2021 = f_score[['Stock', '2021_F_score']]
df_2021.columns = ['Stock', 'F_score_raw']

In [ ]:
final_f = pd.DataFrame(columns = ['Stock', 'F_score_raw'])
final_f = pd.concat([df_2018, df_2019, df_2020, df_2021], axis=0)
final_f.reset_index(drop=True, inplace=True)
final_f


,Stock,F_score_raw
0,389500,4
1,296640,6
2,417970,7
3,314140,6
4,129920,5
...,...,...
5111,950110,4
5112,46970,6
5113,113810,6
5114,155650,7


In [ ]:
baseline
baseline_2 = pd.concat([baseline, pd.DataFrame(final_f['F_score_raw'])],axis=1)
baseline_2

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,F_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,업종,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률,F_score_raw
0,389500,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,2.0,1.0,2.0,0.0,1.0,0.000000,3.0,0.023564,1.0,14.593801,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,0,0,2018,110.399585,-72.542571,263.452137,0.047736,-5.930419,4
1,296640,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,1.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.006766,1.0,17.328700,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2018,90.942256,-63.629994,396.408093,0.470922,-0.354859,6
2,417970,23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,2.0,2.0,2.0,5.0,2.0,0.000000,3.0,0.000343,1.0,26.200702,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,57.427845,-3.584996,595.858592,0.863325,0.549708,7
3,314140,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,1.0,3.0,2.0,3.0,2.0,0.000000,3.0,0.000461,1.0,21.578064,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,6
4,129920,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.001602,1.0,20.705556,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,950110,78.37,-3.08,-3.40,0.92,3.84,34.32,23.90,5.72,17.63,467.08,117.88,0.16,0.70,7.052749e+09,-8.972330e+10,1.0,0.0,1.0,4.0,1.0,0.965841,2.0,0.005466,1.0,15.903376,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2021,110.399585,-72.542571,263.452137,0.047736,-5.930419,4
5112,46970,2.98,99.94,-0.77,5.31,22.63,88.70,13.82,3.24,64.99,53.87,154.51,1.64,3.31,1.335246e+10,-2.713149e+09,1.0,2.0,2.0,4.0,1.0,3.323928,0.0,0.000051,1.0,15.822916,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,1.0,3,1,1,1,2021,4.120000,1.520000,464.000000,0.000000,-2.506494,6
5113,113810,-1.44,-4.48,-0.65,-1.02,0.04,0.16,0.05,-1.31,32.45,208.15,24.23,0.78,2.38,2.625700e+07,2.652719e+09,0.0,1.0,2.0,2.0,2.0,0.806495,2.0,0.002698,1.0,14.801748,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3,1,0,1,2021,7.880000,1.340000,115.000000,0.000000,-1.618280,6
5114,155650,4.66,7.88,22.56,10.32,8.13,89.91,5.03,6.38,80.72,23.88,897.93,1.62,2.03,8.979293e+09,6.462723e+09,2.0,3.0,2.0,3.0,2.0,7.915767,0.0,0.000030,1.0,31.194817,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2021,13.510000,2.130000,525.000000,0.000000,-0.354244,7


In [ ]:
baseline_2['F_score_raw'].value_counts()

6    1132
7    1045
5     844
8     607
4     581
3     452
2     245
9     147
1      59
0       4
Name: F_score_raw, dtype: int64

# 관리종목 지정요건 수치 데이터 추가

In [ ]:

df_for_stock_num = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/REALREALFINALJAM.csv')

관리종목지정요건 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/관리종목지정요건.csv')
관리종목지정요건.head(10)

def make_preprocessing(df):
  col_name = ['KIS', 'Stock', 'Name']
  for i in df.columns[3:]:
    col_name.append(f"{df[i].iloc[5].split('/')[0]} ({df[i].iloc[4]})")


  df.columns = col_name
  df.drop([0,1,2,3,4,5], axis=0, inplace=True)
  df =df.reset_index(drop=True)

  # 뒤에 널값인 데이터도 같이 들어왔는데 같이 삭제
  last_index = df[df['Name']=='(주)아바텍'].index # 데이터가 들어있는 맨 마지막 인덱스. 이 이후꺼를 삭제시키면 된다.
  df = df.iloc[:list(df[df['Name']=='(주)아바텍'].index).pop()+1, :]

  # stock code 없는 기업 삭제
  del_index = list(df[df['Stock'].isnull()].index)
  df.drop(del_index, axis=0, inplace = True)
  df = df.reset_index(drop= True)

  # 인수합병된 기업들 빼기
  df['인수합병여부'] = 0
  df['인수합병여부'] = df.apply(lambda x:1 if '인수목적' in x['Name'] else 0,axis=1)

  df = df[df['인수합병여부']==0]
  df.drop('인수합병여부', axis=1, inplace =True)
  df = df.reset_index(drop=True)




  # 문자열 float로 변경
  for i in df.columns[7:]:
    df[i] = df[i].astype('str')
    df[i] = df[i].apply(lambda x: re.sub(',', '',x))
    df[i] = df[i].astype('float')



  # # null인 경우 0으로 채운다. -> o_score 도출을 위해서(널값이 많이 나오지는 않았다.)
  # for i in df.columns:
  #   if '당기순이익' in i:
  #     df[i].fillna(0, inplace=True)
  #   if '유형,임대주택자산감가상각비' in i:
  #     df[i].fillna(0, inplace=True)
  #   if '무형자산상각비' in i:
  #     df[i].fillna(0, inplace=True)
  #   if '법인세비용 ' in i:
  #     df[i].fillna(0, inplace=True)
  # 종목카드 6자리로 채워주기
  df['Stock'] = df['Stock'].astype('int')
  df['Stock'] = df['Stock'].astype('str')
  df['Stock'] = df['Stock'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

  # 널값 1개 있는것 최빈값으로 채워주기
  df['감사의견 (2020)'].fillna('적정', inplace=True)

  return df

df = make_preprocessing(관리종목지정요건)

In [ ]:

# 종목카드 6자리로 채워주기
df_for_stock_num['Stock'] = df_for_stock_num['Stock'].astype('int')
df_for_stock_num['Stock'] = df_for_stock_num['Stock'].astype('str')
df_for_stock_num['Stock'] = df_for_stock_num['Stock'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

tmp = []
관리종목_df = pd.DataFrame()
for i in df['Stock'].unique():
  if i in df_for_stock_num['Stock'].unique():
    idx = list(df[df['Stock']==i].index.values).pop()
    tmp.append(idx)
관리종목_df  =pd.DataFrame(df, index = tmp).reset_index(drop=True)



관리종목_df

,KIS,Stock,Name,감사의견 (2018),감사의견 (2019),감사의견 (2020),감사의견 (2021),매출액 (2018/Annual),법인세비용차감전계속사업이익(손실) (2018/Annual),영업이익(손실) (2018/Annual),매출액 (2019/Annual),법인세비용차감전계속사업이익(손실) (2019/Annual),영업이익(손실) (2019/Annual),매출액 (2020/Annual),법인세비용차감전계속사업이익(손실) (2020/Annual),영업이익(손실) (2020/Annual),매출액 (2021/Annual),법인세비용차감전계속사업이익(손실) (2021/Annual),영업이익(손실) (2021/Annual)
0,036644,389500,(주)에스비비테크,적정,한정(감사범위제한),적정,적정,9.218357e+09,-1.917403e+09,-1.088044e+09,6.019507e+09,-4.788196e+09,-3.769930e+09,6.606806e+09,-3.309699e+09,-2.616470e+09,6.779379e+09,-2.817270e+09,-2.227144e+09
1,497730,296640,(주)이노룰스,적정,적정,적정,적정,9.313410e+09,1.478880e+08,4.809880e+08,1.340630e+10,1.191682e+09,2.768827e+09,1.063404e+10,1.024106e+09,1.187901e+09,1.597968e+10,3.324217e+09,3.314891e+09
2,499459,417970,모델솔루션(주),적정,적정,적정,적정,5.559806e+10,9.218748e+09,9.433205e+09,6.056418e+10,9.309068e+09,9.332657e+09,5.501846e+10,2.941833e+09,6.870288e+09,6.007888e+10,9.146970e+09,8.526295e+09
3,HS8995,314140,(주)알피바이오,적정,적정,적정,적정,6.476281e+10,5.851262e+09,6.382369e+09,6.985516e+10,3.351179e+09,3.959920e+09,9.792265e+10,4.209396e+09,4.157063e+09,1.149640e+11,4.290818e+09,5.879847e+09
4,867055,129920,(주)대성하이텍,적정,적정,적정,적정,8.863754e+10,2.019903e+09,4.717809e+09,7.314135e+10,3.214249e+09,5.169880e+09,7.265322e+10,-1.170560e+10,-8.452512e+09,1.027162e+11,9.244219e+09,1.169168e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,I73639,950110,SBI핀테크솔루션즈(주),적정,적정,적정,적정,2.374177e+10,3.762250e+09,4.254804e+09,2.144421e+10,1.320041e+09,1.734016e+09,3.055471e+10,8.408703e+09,8.915481e+09,2.951487e+10,1.011052e+10,1.688230e+09
1275,125028,046970,(주)우리로,적정,적정,적정,적정,3.197440e+10,-3.554180e+08,2.718326e+09,1.015362e+11,-8.829858e+09,2.323770e+08,9.735110e+10,-1.162556e+10,-2.502020e+08,9.659889e+10,1.335246e+10,3.130126e+09
1276,383046,113810,디젠스(주),적정,적정,적정,적정,3.723606e+10,-8.257150e+09,-2.946957e+09,3.968948e+10,2.659499e+09,2.632887e+09,4.959402e+10,-7.319745e+09,-3.599090e+08,4.927206e+10,3.523810e+08,-6.435750e+08
1277,F11411,155650,와이엠씨(주),적정,적정,적정,적정,2.315697e+11,1.871956e+10,1.980928e+10,1.623188e+11,-3.341818e+09,-5.169211e+09,1.456117e+11,1.086622e+10,1.025215e+10,1.784667e+11,1.237523e+10,1.139106e+10


In [ ]:
# 감사의견 (2020) 한개만 널값이 나온다.
관리종목_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   KIS                               1279 non-null   object 
 1   Stock                             1279 non-null   object 
 2   Name                              1279 non-null   object 
 3   감사의견 (2018)                       1279 non-null   object 
 4   감사의견 (2019)                       1279 non-null   object 
 5   감사의견 (2020)                       1279 non-null   object 
 6   감사의견 (2021)                       1279 non-null   object 
 7   매출액 (2018/Annual)                 1279 non-null   float64
 8   법인세비용차감전계속사업이익(손실) (2018/Annual)  1279 non-null   float64
 9   영업이익(손실) (2018/Annual)            1279 non-null   float64
 10  매출액 (2019/Annual)                 1279 non-null   float64
 11  법인세비용차감전계속사업이익(손실) (2019/Annual)  1279 non-null   float64
 12  영업이익(손

In [ ]:
df_base = 관리종목_df[['Stock']]
df_2018 = pd.concat([df_base,관리종목_df[[col for col in 관리종목_df.columns if '2018' in col]]], axis=1)
df_2019 = pd.concat([df_base,관리종목_df[[col for col in 관리종목_df.columns if '2019' in col]]], axis=1)
df_2020 = pd.concat([df_base,관리종목_df[[col for col in 관리종목_df.columns if '2020' in col]]], axis=1)
df_2021 = pd.concat([df_base,관리종목_df[[col for col in 관리종목_df.columns if '2021' in col]]], axis=1)

In [ ]:
df_2018.columns = ['Stock_2', '감사의견', '매출액', '법인세비용차감전계속사업이익(손실)', '영업이익(손실)']
df_2019.columns = ['Stock_2', '감사의견', '매출액', '법인세비용차감전계속사업이익(손실)', '영업이익(손실)']
df_2020.columns = ['Stock_2', '감사의견', '매출액', '법인세비용차감전계속사업이익(손실)', '영업이익(손실)']
df_2021.columns = ['Stock_2', '감사의견', '매출액', '법인세비용차감전계속사업이익(손실)', '영업이익(손실)']

In [ ]:
관리종목지정요건_수치 = pd.concat([df_2018, df_2019, df_2020, df_2021], axis=0)
관리종목지정요건_수치.reset_index(drop=True, inplace=True)
관리종목지정요건_수치

,Stock_2,감사의견,매출액,법인세비용차감전계속사업이익(손실),영업이익(손실)
0,389500,적정,9.218357e+09,-1.917403e+09,-1.088044e+09
1,296640,적정,9.313410e+09,1.478880e+08,4.809880e+08
2,417970,적정,5.559806e+10,9.218748e+09,9.433205e+09
3,314140,적정,6.476281e+10,5.851262e+09,6.382369e+09
4,129920,적정,8.863754e+10,2.019903e+09,4.717809e+09
...,...,...,...,...,...
5111,950110,적정,2.951487e+10,1.011052e+10,1.688230e+09
5112,046970,적정,9.659889e+10,1.335246e+10,3.130126e+09
5113,113810,적정,4.927206e+10,3.523810e+08,-6.435750e+08
5114,155650,적정,1.784667e+11,1.237523e+10,1.139106e+10


In [ ]:
baseline_2 = pd.concat([baseline_2, 관리종목지정요건_수치], axis=1)
baseline_2

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,F_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,업종,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률,F_score_raw,Stock_2,감사의견,매출액,법인세비용차감전계속사업이익(손실),영업이익(손실)
0,389500,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,2.0,1.0,2.0,0.0,1.0,0.000000,3.0,0.023564,1.0,14.593801,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,0,0,2018,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,389500,적정,9.218357e+09,-1.917403e+09,-1.088044e+09
1,296640,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,1.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.006766,1.0,17.328700,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2018,90.942256,-63.629994,396.408093,0.470922,-0.354859,6,296640,적정,9.313410e+09,1.478880e+08,4.809880e+08
2,417970,23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,2.0,2.0,2.0,5.0,2.0,0.000000,3.0,0.000343,1.0,26.200702,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,57.427845,-3.584996,595.858592,0.863325,0.549708,7,417970,적정,5.559806e+10,9.218748e+09,9.433205e+09
3,314140,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,1.0,3.0,2.0,3.0,2.0,0.000000,3.0,0.000461,1.0,21.578064,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,6,314140,적정,6.476281e+10,5.851262e+09,6.382369e+09
4,129920,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.001602,1.0,20.705556,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,5,129920,적정,8.863754e+10,2.019903e+09,4.717809e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,950110,78.37,-3.08,-3.40,0.92,3.84,34.32,23.90,5.72,17.63,467.08,117.88,0.16,0.70,7.052749e+09,-8.972330e+10,1.0,0.0,1.0,4.0,1.0,0.965841,2.0,0.005466,1.0,15.903376,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2021,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,950110,적정,2.951487e+10,1.011052e+10,1.688230e+09
5112,46970,2.98,99.94,-0.77,5.31,22.63,88.70,13.82,3.24,64.99,53.87,154.51,1.64,3.31,1.335246e+10,-2.713149e+09,1.0,2.0,2.0,4.0,1.0,3.323928,0.0,0.000051,1.0,15.822916,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,1.0,3,1,1,1,2021,4.120000,1.520000,464.000000,0.000000,-2.506494,6,046970,적정,9.659889e+10,1.335246e+10,3.130126e+09
5113,113810,-1.44,-4.48,-0.65,-1.02,0.04,0.16,0.05,-1.31,32.45,208.15,24.23,0.78,2.38,2.625700e+07,2.652719e+09,0.0,1.0,2.0,2.0,2.0,0.806495,2.0,0.002698,1.0,14.801748,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3,1,0,1,2021,7.880000,1.340000,115.000000,0.000000,-1.618280,6,113810,적정,4.927206e+10,3.523810e+08,-6.435750e+08
5114,155650,4.66,7.88,22.56,10.32,8.13,89.91,5.03,6.38,80.72,23.88,897.93,1.62,2.03,8.979293e+09,6.462723e+09,2.0,3.0,2.0,3.0,2.0,7.915767,0.0,0.000030,1.0,31.194817,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2021,13.510000,2.130000,525.000000,0.000000,-0.354244,7,155650,적정,1.784667e+11,1.237523e+10,1.139106e+10


In [ ]:
# 혹시나 종목 어긋낫는지 확인

baseline_2['Stock'] = baseline_2['Stock'].astype('str')
baseline_2['Stock'] = baseline_2['Stock'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

baseline_2['Stock_2'] = baseline_2['Stock_2'].astype('str')
baseline_2['Stock_2'] = baseline_2['Stock_2'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

tmp = []
for i in range(len(baseline_2)):
  if baseline_2['Stock'][i]!= baseline_2['Stock_2'][i]:
    tmp.append(i)
tmp


[]

In [ ]:
baseline_2.drop(['Stock_2', '감사의견'], axis=1, inplace=True)
baseline_2

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,F_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,업종,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률,F_score_raw,매출액,법인세비용차감전계속사업이익(손실),영업이익(손실)
0,389500,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,2.0,1.0,2.0,0.0,1.0,0.000000,3.0,0.023564,1.0,14.593801,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,0,0,2018,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,9.218357e+09,-1.917403e+09,-1.088044e+09
1,296640,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,1.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.006766,1.0,17.328700,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2018,90.942256,-63.629994,396.408093,0.470922,-0.354859,6,9.313410e+09,1.478880e+08,4.809880e+08
2,417970,23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,2.0,2.0,2.0,5.0,2.0,0.000000,3.0,0.000343,1.0,26.200702,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,57.427845,-3.584996,595.858592,0.863325,0.549708,7,5.559806e+10,9.218748e+09,9.433205e+09
3,314140,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,1.0,3.0,2.0,3.0,2.0,0.000000,3.0,0.000461,1.0,21.578064,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,6,6.476281e+10,5.851262e+09,6.382369e+09
4,129920,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.001602,1.0,20.705556,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,5,8.863754e+10,2.019903e+09,4.717809e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,950110,78.37,-3.08,-3.40,0.92,3.84,34.32,23.90,5.72,17.63,467.08,117.88,0.16,0.70,7.052749e+09,-8.972330e+10,1.0,0.0,1.0,4.0,1.0,0.965841,2.0,0.005466,1.0,15.903376,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2021,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,2.951487e+10,1.011052e+10,1.688230e+09
5112,046970,2.98,99.94,-0.77,5.31,22.63,88.70,13.82,3.24,64.99,53.87,154.51,1.64,3.31,1.335246e+10,-2.713149e+09,1.0,2.0,2.0,4.0,1.0,3.323928,0.0,0.000051,1.0,15.822916,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,1.0,3,1,1,1,2021,4.120000,1.520000,464.000000,0.000000,-2.506494,6,9.659889e+10,1.335246e+10,3.130126e+09
5113,113810,-1.44,-4.48,-0.65,-1.02,0.04,0.16,0.05,-1.31,32.45,208.15,24.23,0.78,2.38,2.625700e+07,2.652719e+09,0.0,1.0,2.0,2.0,2.0,0.806495,2.0,0.002698,1.0,14.801748,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3,1,0,1,2021,7.880000,1.340000,115.000000,0.000000,-1.618280,6,4.927206e+10,3.523810e+08,-6.435750e+08
5114,155650,4.66,7.88,22.56,10.32,8.13,89.91,5.03,6.38,80.72,23.88,897.93,1.62,2.03,8.979293e+09,6.462723e+09,2.0,3.0,2.0,3.0,2.0,7.915767,0.0,0.000030,1.0,31.194817,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2021,13.510000,2.130000,525.000000,0.000000,-0.354244,7,1.784667e+11,1.237523e+10,1.139106e+10


# 기업정보 크롤링

In [ ]:
corp_info  = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/코스닥_기업정보.csv')
corp_info['종목코드'] = corp_info['종목코드'].astype('str')
corp_info['종목코드'] = corp_info['종목코드'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기
corp_info

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,000210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시
1,DRB동일,004840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
3,GS,078930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
4,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"최익훈, 정익희, 김회언 (각자 대표이사)",http://www.hdc-dvp.com,서울특별시
...,...,...,...,...,...,...,...,...,...
2544,코스텍시스템,169670,특수 목적용 기계 제조업,"반도체 웨이퍼 이송장비, 본딩장비",2022-01-21,12월,배준호,http://www.kosteks.com/,경기도
2545,타임기술,318660,소프트웨어 개발 및 공급업,"ILS(종합군수지원), IETM(전자식기술교범), CBT(전자식교보재), 기술번역",2021-06-24,12월,주양효,http://timett.co.kr,경상남도
2546,플럼라인생명과학,222670,자연과학 및 공학 연구개발업,"강아지암치료제,돼지DNA테라피",2015-07-28,12월,김앤토니경태,http://www.plsdna.com,서울특별시
2547,한국미라클피플사,331660,기타 화학제품 제조업,"은나노스텝, 닥터오렌지 등",2019-10-28,12월,이호경,http://www.kmpc.co.kr,경기도


In [ ]:

info_idx = []
for i in list(corp_info['종목코드'].unique()):
  if i in list(baseline_2['Stock'].unique()):
    idx = list(corp_info[corp_info['종목코드']==i].index).pop()
    info_idx.append(idx)




In [ ]:
idx

2530

In [ ]:
new_corp_info = corp_info.loc[info_idx]
new_corp_info

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
100,CJ 바이오사이언스,311690,자연과학 및 공학 연구개발업,"생명정보 플랫폼, 마이크로바이옴 분석 서비스",2019-12-26,12월,천종식,http://www.cjbioscience.com,서울특별시
101,ES큐브,050120,생활용품 도매업,"텐트, 원 ·부자재",2002-04-25,12월,신희민,http://www.escube.co.kr,서울특별시
102,HPSP,403870,특수 목적용 기계 제조업,고압 수소 어닐링 장비,2022-07-15,12월,김용운,http://www.thehpsp.com/,경기도
104,ISC,095340,전자부품 제조업,반도체테스트소켓,2007-10-01,12월,"김정렬, 김상욱 (각자 대표이사)",http://isc21.kr,경기도
105,JW신약,067290,의약품 제조업,"매니큐어세트, 손톱깎이",2003-02-11,12월,백승호,http://www.cwsy.co.kr,서울특별시
...,...,...,...,...,...,...,...,...,...
2526,한컴위드,054920,소프트웨어 개발 및 공급업,전자상거래 보안 솔루션,2001-11-01,12월,"김현수, 송상엽(각자대표)",http://www.hsecure.co.kr,경기도
2527,한탑,002680,"곡물가공품, 전분 및 전분제품 제조업","소맥분(밀가루),배합사료",1995-07-07,12월,하상경,http://www.hantop.com,부산광역시
2528,헥토이노베이션,214180,기타 정보 서비스업,휴대폰부가서비스,2015-06-30,12월,이현철,http://www.hectoinnovation.co.kr,서울특별시
2529,홈센타홀딩스,060560,기타 전문 도매업,"건축자재,레미콘",2002-07-11,12월,박 병 윤,http://www.home-center.co.kr,대구광역시


In [ ]:
# merge
baseline_3 = pd.merge(baseline_2, new_corp_info, how='left', left_on = 'Stock', right_on='종목코드')
baseline_3.drop('종목코드', axis=1, inplace=True)
baseline_3.rename(columns = {'업종_y':'업종설명'}, inplace=True)
baseline_3


,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,F_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,업종_x,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률,F_score_raw,매출액,법인세비용차감전계속사업이익(손실),영업이익(손실),회사명,업종설명,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,389500,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,2.0,1.0,2.0,0.0,1.0,0.000000,3.0,0.023564,1.0,14.593801,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,0,0,2018,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,9.218357e+09,-1.917403e+09,-1.088044e+09,에스비비테크,일반 목적용 기계 제조업,"베어링, 감속기",2022-10-17,12월,류재완,http://www.sbb.co.kr/,경기도
1,296640,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,1.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.006766,1.0,17.328700,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2018,90.942256,-63.629994,396.408093,0.470922,-0.354859,6,9.313410e+09,1.478880e+08,4.809880e+08,이노룰스,소프트웨어 개발 및 공급업,"InnoRules, InnoProduct",2022-10-07,12월,김길곤,http://innorules.com,서울특별시
2,417970,23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,2.0,2.0,2.0,5.0,2.0,0.000000,3.0,0.000343,1.0,26.200702,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,57.427845,-3.584996,595.858592,0.863325,0.549708,7,5.559806e+10,9.218748e+09,9.433205e+09,모델솔루션,특수 목적용 기계 제조업,"하이엔드 프로토타입 제조, 다품종 소량생산 서비스",2022-10-07,12월,우병일,http://www.model-solution.com,서울특별시
3,314140,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,1.0,3.0,2.0,3.0,2.0,0.000000,3.0,0.000461,1.0,21.578064,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,6,6.476281e+10,5.851262e+09,6.382369e+09,알피바이오,의약품 제조업,연질캡슐 제형의 의약품 및 건강기능식품,2022-09-29,12월,"윤재훈, 김남기",http://rpskorea.com,서울특별시
4,129920,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.001602,1.0,20.705556,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,5,8.863754e+10,2.019903e+09,4.717809e+09,대성하이텍,특수 목적용 기계 제조업,"정밀부품, 스위스턴 자동선반, 컴팩트 머시닝센터",2022-08-22,12월,"최우각, 최호형",http:///www.topdsht.com/,대구광역시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,950110,78.37,-3.08,-3.40,0.92,3.84,34.32,23.90,5.72,17.63,467.08,117.88,0.16,0.70,7.052749e+09,-8.972330e+10,1.0,0.0,1.0,4.0,1.0,0.965841,2.0,0.005466,1.0,15.903376,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2021,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,2.951487e+10,1.011052e+10,1.688230e+09,SBI핀테크솔루션즈,소프트웨어 개발 및 공급업,전자결제대행 및 핀테크 비스,2012-12-17,03월,카네코 유이치,http://https://www.sbi-finsol.co.jp/kr/,일본
5112,046970,2.98,99.94,-0.77,5.31,22.63,88.70,13.82,3.24,64.99,53.87,154.51,1.64,3.31,1.335246e+10,-2.713149e+09,1.0,2.0,2.0,4.0,1.0,3.323928,0.0,0.000051,1.0,15.822916,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,1.0,3,1,1,1,2021,4.120000,1.520000,464.000000,0.000000,-2.506494,6,9.659889e+10,1.335246e+10,3.130126e+09,우리로,기타 전문 도매업,"광분배기(웨이퍼, 칩, 모듈), 광다이오드",2012-11-27,12월,박세철,http://www.wooriro.com,광주광역시
5113,113810,-1.44,-4.48,-0.65,-1.02,0.04,0.16,0.05,-1.31,32.45,208.15,24.23,0.78,2.38,2.625700e+07,2.652719e+09,0.0,1.0,2.0,2.0,2.0,0.806495,2.0,0.002698,1.0,14.801748,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3,1,0,1,2021,7.880000,1.340000,115.000000,0.000000,-1.618280,6,4.927206e+10,3.523810e+08,-6.435750e+08,디젠스,자동차 신품 부품 제조업,자동차용 소음기(머플러),2012-11-22,12월,이석우,http://www.dgenx.com,경기도
5114,155650,4.66,7.88,22.56,10.32,8.13,89.91,5.03,6.38,80.72,23.88,897.93,1.62,2.03,8.97929

In [ ]:
baseline_3.columns

Index(['Stock', '총자산증가율', '자기자본증가율', '매출액증가율', '총자본영업이익율', '총자본순이익율',
       '자본금순이익율', '매출액순이익율', '매출액영업이익율', '자기자본비율', '부채비율', '유보액/납입자본비율',
       '총자본회전율', '자기자본회전율', ' 당기순이익(손실)', '영업활동으로인한현금흐름', '성장성', '안정성', '활동성',
       '수익성', '현금흐름', 'Z_score', '부실기업판별_Z_score', '부도확률_O_score', 'F_score',
       'K_score', '부실기업판별_K_score', '자본잠식', '벌금', '불성실공시', '소송', '영업정지', '특허',
       '투자주의환기종목', '업종_x', '관리종목여부', '관리종목요건_감사의견', '관리종목요건_매출액',
       '관리종목요건_영업손실', '관리종목요건_법인세비용차감전계속사업손실', '날짜', 'PER', 'PBR', 'EPS',
       'DIV', 'EPS성장률', 'F_score_raw', '매출액', '법인세비용차감전계속사업이익(손실)', '영업이익(손실)',
       '회사명', '업종설명', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역'],
      dtype='object')

In [ ]:
baseline_3['부실기업판별_K_score'].value_counts()

2.0    4932
0.0     148
1.0      36
Name: 부실기업판별_K_score, dtype: int64

# 업종 replace

In [ ]:
업종 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/업종파일.csv')
업종

,KIS,Stock,Name_x,업종,한자리
0,36644,389500,(주)에스비비테크,C29,9
1,497730,296640,(주)이노룰스,J58,0
2,499459,417970,모델솔루션(주),C29,9
3,HS8995,314140,(주)알피바이오,C21,6
4,867055,129920,(주)대성하이텍,C29,9
...,...,...,...,...,...
1274,I73639,950110,SBI핀테크솔루션즈(주),J58,0
1275,125028,46970,(주)우리로,G46,3
1276,383046,113810,디젠스(주),C30,10
1277,F11411,155650,와이엠씨(주),C29,9


In [ ]:
업종 = 업종[['Stock', '업종', '한자리']]
업종.columns = ['Stock_3', '업종_세분화', '업종_label']
업종['Stock_3'] = 업종['Stock_3'].astype('str')
업종['Stock_3'] = 업종['Stock_3'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

In [ ]:
업종

,Stock_3,업종_세분화,업종_label
0,389500,C29,9
1,296640,J58,0
2,417970,C29,9
3,314140,C21,6
4,129920,C29,9
...,...,...,...
1274,950110,J58,0
1275,046970,G46,3
1276,113810,C30,10
1277,155650,C29,9


In [ ]:
baseline_4 = pd.merge(baseline_3, 업종, how='left', left_on = 'Stock', right_on = 'Stock_3')
baseline_4

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,F_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,업종_x,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률,F_score_raw,매출액,법인세비용차감전계속사업이익(손실),영업이익(손실),회사명,업종설명,주요제품,상장일,결산월,대표자명,홈페이지,지역,Stock_3,업종_세분화,업종_label
0,389500,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,2.0,1.0,2.0,0.0,1.0,0.000000,3.0,0.023564,1.0,14.593801,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,0,0,2018,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,9.218357e+09,-1.917403e+09,-1.088044e+09,에스비비테크,일반 목적용 기계 제조업,"베어링, 감속기",2022-10-17,12월,류재완,http://www.sbb.co.kr/,경기도,389500,C29,9
1,296640,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,1.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.006766,1.0,17.328700,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2018,90.942256,-63.629994,396.408093,0.470922,-0.354859,6,9.313410e+09,1.478880e+08,4.809880e+08,이노룰스,소프트웨어 개발 및 공급업,"InnoRules, InnoProduct",2022-10-07,12월,김길곤,http://innorules.com,서울특별시,296640,J58,0
2,417970,23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,2.0,2.0,2.0,5.0,2.0,0.000000,3.0,0.000343,1.0,26.200702,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,57.427845,-3.584996,595.858592,0.863325,0.549708,7,5.559806e+10,9.218748e+09,9.433205e+09,모델솔루션,특수 목적용 기계 제조업,"하이엔드 프로토타입 제조, 다품종 소량생산 서비스",2022-10-07,12월,우병일,http://www.model-solution.com,서울특별시,417970,C29,9
3,314140,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,1.0,3.0,2.0,3.0,2.0,0.000000,3.0,0.000461,1.0,21.578064,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,6,6.476281e+10,5.851262e+09,6.382369e+09,알피바이오,의약품 제조업,연질캡슐 제형의 의약품 및 건강기능식품,2022-09-29,12월,"윤재훈, 김남기",http://rpskorea.com,서울특별시,314140,C21,6
4,129920,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.001602,1.0,20.705556,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,5,8.863754e+10,2.019903e+09,4.717809e+09,대성하이텍,특수 목적용 기계 제조업,"정밀부품, 스위스턴 자동선반, 컴팩트 머시닝센터",2022-08-22,12월,"최우각, 최호형",http:///www.topdsht.com/,대구광역시,129920,C29,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,950110,78.37,-3.08,-3.40,0.92,3.84,34.32,23.90,5.72,17.63,467.08,117.88,0.16,0.70,7.052749e+09,-8.972330e+10,1.0,0.0,1.0,4.0,1.0,0.965841,2.0,0.005466,1.0,15.903376,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2021,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,2.951487e+10,1.011052e+10,1.688230e+09,SBI핀테크솔루션즈,소프트웨어 개발 및 공급업,전자결제대행 및 핀테크 비스,2012-12-17,03월,카네코 유이치,http://https://www.sbi-finsol.co.jp/kr/,일본,950110,J58,0
5112,046970,2.98,99.94,-0.77,5.31,22.63,88.70,13.82,3.24,64.99,53.87,154.51,1.64,3.31,1.335246e+10,-2.713149e+09,1.0,2.0,2.0,4.0,1.0,3.323928,0.0,0.000051,1.0,15.822916,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,1.0,3,1,1,1,2021,4.120000,1.520000,464.000000,0.000000,-2.506494,6,9.659889e+10,1.335246e+10,3.130126e+09,우리로,기타 전문 도매업,"광분배기(웨이퍼, 칩, 모듈), 광다이오드",2012-11-27,12월,박세철,http://www.wooriro.com,광주광역시,046970,G46,3
5113,113810,-1.44,-4.48,-0.65,-1.02,0.04,0.16,0.05,-1.31,32.45,208.15,24.23,0.78,2.38,2.625700e+07,2.652719e+09,0.0,1.0,2.0,2.0,2.0,0.806495,2.0,0.002698,1.0,14.801748,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3,1,0,1,2021,7.880000,1.340000,115.000000,0.000000,-1.618280,6,4.927206e+10,3.523810e+08,-6.435750e+08,디젠스,자동차 신품 부품 제조업,자동차용 소음기(머플러),2012-11-

In [ ]:
baseline_4.drop('Stock_3', axis=1, inplace=True)
baseline_4.reset_index(drop=True)

baseline_4.to_csv('data_for_visualization.csv', index= False)

In [ ]:
baseline_3

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,F_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,업종_x,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률,F_score_raw,매출액,법인세비용차감전계속사업이익(손실),영업이익(손실),회사명,업종설명,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,389500,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,2.0,1.0,2.0,0.0,1.0,0.000000,3.0,0.023564,1.0,14.593801,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,0,0,2018,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,9.218357e+09,-1.917403e+09,-1.088044e+09,에스비비테크,일반 목적용 기계 제조업,"베어링, 감속기",2022-10-17,12월,류재완,http://www.sbb.co.kr/,경기도
1,296640,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,1.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.006766,1.0,17.328700,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2018,90.942256,-63.629994,396.408093,0.470922,-0.354859,6,9.313410e+09,1.478880e+08,4.809880e+08,이노룰스,소프트웨어 개발 및 공급업,"InnoRules, InnoProduct",2022-10-07,12월,김길곤,http://innorules.com,서울특별시
2,417970,23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,2.0,2.0,2.0,5.0,2.0,0.000000,3.0,0.000343,1.0,26.200702,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,57.427845,-3.584996,595.858592,0.863325,0.549708,7,5.559806e+10,9.218748e+09,9.433205e+09,모델솔루션,특수 목적용 기계 제조업,"하이엔드 프로토타입 제조, 다품종 소량생산 서비스",2022-10-07,12월,우병일,http://www.model-solution.com,서울특별시
3,314140,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,1.0,3.0,2.0,3.0,2.0,0.000000,3.0,0.000461,1.0,21.578064,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,6,6.476281e+10,5.851262e+09,6.382369e+09,알피바이오,의약품 제조업,연질캡슐 제형의 의약품 및 건강기능식품,2022-09-29,12월,"윤재훈, 김남기",http://rpskorea.com,서울특별시
4,129920,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.001602,1.0,20.705556,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1,2018,77.101658,-20.206862,577.005958,0.815275,-0.072114,5,8.863754e+10,2.019903e+09,4.717809e+09,대성하이텍,특수 목적용 기계 제조업,"정밀부품, 스위스턴 자동선반, 컴팩트 머시닝센터",2022-08-22,12월,"최우각, 최호형",http:///www.topdsht.com/,대구광역시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,950110,78.37,-3.08,-3.40,0.92,3.84,34.32,23.90,5.72,17.63,467.08,117.88,0.16,0.70,7.052749e+09,-8.972330e+10,1.0,0.0,1.0,4.0,1.0,0.965841,2.0,0.005466,1.0,15.903376,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1,2021,110.399585,-72.542571,263.452137,0.047736,-5.930419,4,2.951487e+10,1.011052e+10,1.688230e+09,SBI핀테크솔루션즈,소프트웨어 개발 및 공급업,전자결제대행 및 핀테크 비스,2012-12-17,03월,카네코 유이치,http://https://www.sbi-finsol.co.jp/kr/,일본
5112,046970,2.98,99.94,-0.77,5.31,22.63,88.70,13.82,3.24,64.99,53.87,154.51,1.64,3.31,1.335246e+10,-2.713149e+09,1.0,2.0,2.0,4.0,1.0,3.323928,0.0,0.000051,1.0,15.822916,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,1.0,3,1,1,1,2021,4.120000,1.520000,464.000000,0.000000,-2.506494,6,9.659889e+10,1.335246e+10,3.130126e+09,우리로,기타 전문 도매업,"광분배기(웨이퍼, 칩, 모듈), 광다이오드",2012-11-27,12월,박세철,http://www.wooriro.com,광주광역시
5113,113810,-1.44,-4.48,-0.65,-1.02,0.04,0.16,0.05,-1.31,32.45,208.15,24.23,0.78,2.38,2.625700e+07,2.652719e+09,0.0,1.0,2.0,2.0,2.0,0.806495,2.0,0.002698,1.0,14.801748,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3,1,0,1,2021,7.880000,1.340000,115.000000,0.000000,-1.618280,6,4.927206e+10,3.523810e+08,-6.435750e+08,디젠스,자동차 신품 부품 제조업,자동차용 소음기(머플러),2012-11-22,12월,이석우,http://www.dgenx.com,경기도
5114,155650,4.66,7.88,22.56,10.32,8.13,89.91,5.03,6.38,80.72,23.88,897.93,1.62,2.03,8.97929

In [ ]:
corp_name= pd.read_csv('/content/drive/MyDrive/핀테크 파이널/부실지표/altman.csv')
corp_name

def del_x(x):
  x= x.replace('(주)','')
  return x

corp_name['Stock'] = corp_name['Stock'].astype('str')
corp_name['Stock'] = corp_name['Stock'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기
corp_name['Name'] = corp_name['Name'].apply(del_x)
for_crawling = corp_name[['Stock', 'Name']]
for_crawling.to_csv('Stock_Name.csv', index= False)

# 성은누나 df 수정
- z score 수정

In [4]:
df = pd.read_excel('/content/drive/MyDrive/핀테크 파이널/(성은) 부실기업판별_Z_score (w열).xlsx')
df.head(3)

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,1,부도확률_O_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률,F_score_raw,매출액,법인세비용차감전계속사업이익(손실),영업이익(손실),회사명,업종설명,주요제품,상장일,결산월,대표자명,홈페이지,지역,업종_세분화,업종_label
0,평균,18.681923,35.363958,16.939203,3.707248,1.01484,124.411859,-16.248642,-10.247494,65.018069,81.617527,1521.698501,0.739408,1.398059,4.958299e+09,8.741003e+09,77.586656,88.316067,95.901746,59.212927,79.599948,6.208198,NaN,0.010208,21.240447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.188981,-41.177780,587.025438,0.668026,-1.057112,5.616693,1.128886e+11,6.492112e+09,7.490148e+09,평균,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,389500,46.030000,-39.460000,2.260000,-5.230000,-9.21000,-168.760000,-20.800000,-11.800000,20.220000,394.440000,340.070000,0.440000,1.390000,-1.917031e+09,6.075800e+07,100.000000,50.000000,100.000000,16.666667,66.666667,0.000000,3.0,0.023564,14.593801,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,불안정,불안정,2018.0,110.399585,-72.542571,263.452137,0.047736,-5.930419,4.000000,9.218357e+09,-1.917403e+09,-1.088044e+09,에스비비테크,일반 목적용 기계 제조업,"베어링, 감속기",2022-10-17,12월,류재완,http://www.sbb.co.kr/,경기도,C29,9.0
2,296640,81.840000,215.020000,-6.230000,3.880000,1.43000,68.580000,1.900000,5.160000,40.950000,144.170000,2371.700000,0.750000,2.160000,1.770280e+08,1.148444e+09,66.666667,75.000000,100.000000,66.666667,100.000000,0.000000,3.0,0.006766,17.328700,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,안정,안정,2018.0,90.942256,-63.629994,396.408093,0.470922,-0.354859,6.000000,9.313410e+09,1.478880e+08,4.809880e+08,이노룰스,소프트웨어 개발 및 공급업,"InnoRules, InnoProduct",2022-10-07,12월,김길곤,http://innorules.com,서울특별시,J58,0.0


In [11]:
df.rename(columns = {1 : '부실기업판별_Z_score'}, inplace = True)
df

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률,F_score_raw,매출액,법인세비용차감전계속사업이익(손실),영업이익(손실),회사명,업종설명,주요제품,상장일,결산월,대표자명,홈페이지,지역,업종_세분화,업종_label
0,평균,18.681923,35.363958,16.939203,3.707248,1.01484,124.411859,-16.248642,-10.247494,65.018069,81.617527,1521.698501,0.739408,1.398059,4.958299e+09,8.741003e+09,77.586656,88.316067,95.901746,59.212927,79.599948,6.208198,NaN,0.010208,21.240447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.188981,-41.177780,587.025438,0.668026,-1.057112,5.616693,1.128886e+11,6.492112e+09,7.490148e+09,평균,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,389500,46.030000,-39.460000,2.260000,-5.230000,-9.21000,-168.760000,-20.800000,-11.800000,20.220000,394.440000,340.070000,0.440000,1.390000,-1.917031e+09,6.075800e+07,100.000000,50.000000,100.000000,16.666667,66.666667,0.000000,3.0,0.023564,14.593801,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,불안정,불안정,2018.0,110.399585,-72.542571,263.452137,0.047736,-5.930419,4.000000,9.218357e+09,-1.917403e+09,-1.088044e+09,에스비비테크,일반 목적용 기계 제조업,"베어링, 감속기",2022-10-17,12월,류재완,http://www.sbb.co.kr/,경기도,C29,9.0
2,296640,81.840000,215.020000,-6.230000,3.880000,1.43000,68.580000,1.900000,5.160000,40.950000,144.170000,2371.700000,0.750000,2.160000,1.770280e+08,1.148444e+09,66.666667,75.000000,100.000000,66.666667,100.000000,0.000000,3.0,0.006766,17.328700,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,안정,안정,2018.0,90.942256,-63.629994,396.408093,0.470922,-0.354859,6.000000,9.313410e+09,1.478880e+08,4.809880e+08,이노룰스,소프트웨어 개발 및 공급업,"InnoRules, InnoProduct",2022-10-07,12월,김길곤,http://innorules.com,서울특별시,J58,0.0
3,417970,23.530000,44.460000,9.650000,25.300000,19.34000,768.480000,12.970000,16.970000,47.940000,108.600000,2005.450000,1.490000,3.330000,7.210181e+09,5.469434e+09,100.000000,75.000000,100.000000,100.000000,100.000000,0.000000,3.0,0.000343,26.200702,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,안정,안정,2018.0,57.427845,-3.584996,595.858592,0.863325,0.549708,7.000000,5.559806e+10,9.218748e+09,9.433205e+09,모델솔루션,특수 목적용 기계 제조업,"하이엔드 프로토타입 제조, 다품종 소량생산 서비스",2022-10-07,12월,우병일,http://www.model-solution.com,서울특별시,C29,9.0
4,314140,38.750000,31.690000,-5.020000,8.280000,5.66000,146.000000,6.740000,9.850000,58.820000,70.000000,1558.220000,0.840000,1.400000,4.363446e+09,6.995499e+09,66.666667,100.000000,100.000000,66.666667,100.000000,0.000000,3.0,0.000461,21.578064,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,안정,안정,2018.0,77.101658,-20.206862,577.005958,0.815275,-0.072114,6.000000,6.476281e+10,5.851262e+09,6.382369e+09,알피바이오,의약품 제조업,연질캡슐 제형의 의약품 및 건강기능식품,2022-09-29,12월,"윤재훈, 김남기",http://rpskorea.com,서울특별시,C21,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,950110,78.370000,-3.080000,-3.400000,0.920000,3.84000,34.320000,23.900000,5.720000,17.630000,467.080000,117.880000,0.160000,0.700000,7.052749e+09,-8.972330e+10,66.666667,25.000000,66.666667,83.333333,66.666667,0.965841,2.0,0.005466,15.903376,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,안정,안정,2021.0,110.399585,-72.542571,263.452137,0.047736,-5.930419,4.000000,2.951487e+10,1.011052e+10,1.688230e+09,SBI핀테크솔루션즈,소프트웨어 개발 및 공급업,전자결제대행 및 핀테크 비스,2012-12-17,03월,카네코 유이치,http://https://www.sbi-finsol.co.jp/kr/,일본,J58,0.0
5113,46970,2.980000,99.940000,-0.770000,5.310000,22.63000,88.700000,13.820000,3.240000,64.990000,53.870000,154.510000,1.640000,3.310000,1.335246e+10,-2.713149e+09,66.666667,75.000000,100.000000,83.333333,66.666667,3.323928,0.0,0.000051,15.822916,안전,비해당,비해당,비해당,해당,비해당,해당,해당,관리종목,적정,안정,안정,안정,2021.0,4.120000,1.520000,464.000000,0.000000,-2.506494,6.000000,9.659

In [6]:
altman = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/altman (4).csv')
altman

,Stock,Name,2018_Z,2019_Z,2020_Z,2021_Z,2018_부실기업판별,2019_부실기업판별,2020_부실기업판별,2021_부실기업판별
0,389500,(주)에스비비테크,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,296640,(주)이노룰스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,417970,모델솔루션(주),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,314140,(주)알피바이오,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,129920,(주)대성하이텍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1274,950110,SBI핀테크솔루션즈(주),2.995899,3.942799,2.596882,0.965841,건전 단계 기업,안전 단계 기업,예의주시 단계 기업,경계태세 단계 기업
1275,46970,(주)우리로,1.289412,1.650250,1.154469,3.323928,경계태세 단계 기업,경계태세 단계 기업,경계태세 단계 기업,안전 단계 기업
1276,113810,디젠스(주),0.494274,0.746953,0.601730,0.806495,경계태세 단계 기업,경계태세 단계 기업,경계태세 단계 기업,경계태세 단계 기업
1277,155650,와이엠씨(주),3.506547,3.499731,7.871761,7.915767,안전 단계 기업,안전 단계 기업,안전 단계 기업,안전 단계 기업


In [17]:
df_2018 = altman[['Stock', '2018_부실기업판별']]
df_2018.columns = ['Stock', 'z_부실기업판별']

df_2019 = altman[['Stock', '2019_부실기업판별']]
df_2019.columns = ['Stock', 'z_부실기업판별']

df_2020 = altman[['Stock', '2020_부실기업판별']]
df_2020.columns = ['Stock', 'z_부실기업판별']

df_2021 = altman[['Stock', '2021_부실기업판별']]
df_2021.columns = ['Stock', 'z_부실기업판별']

altman_df = pd.concat([df_2018, df_2019, df_2020, df_2021], axis=0)
altman_df.reset_index(drop=True, inplace=True)
altman_df

,Stock,z_부실기업판별
0,389500,NaN
1,296640,NaN
2,417970,NaN
3,314140,NaN
4,129920,NaN
...,...,...
5111,950110,경계태세 단계 기업
5112,46970,안전 단계 기업
5113,113810,경계태세 단계 기업
5114,155650,안전 단계 기업


In [19]:

new_row = pd.DataFrame([[np.nan, np.nan]], columns = altman_df.columns)
altman_df2 = pd.concat([new_row, altman_df], axis=0)
altman_df2
altman_df2.reset_index(drop=True, inplace= True)
altman_df2

,Stock,z_부실기업판별
0,NaN,NaN
1,389500.0,NaN
2,296640.0,NaN
3,417970.0,NaN
4,314140.0,NaN
...,...,...
5112,950110.0,경계태세 단계 기업
5113,46970.0,안전 단계 기업
5114,113810.0,경계태세 단계 기업
5115,155650.0,안전 단계 기업


In [21]:
df['부실기업판별_Z_score'] = altman_df2['z_부실기업판별']
df

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실,날짜,PER,PBR,EPS,DIV,EPS성장률,F_score_raw,매출액,법인세비용차감전계속사업이익(손실),영업이익(손실),회사명,업종설명,주요제품,상장일,결산월,대표자명,홈페이지,지역,업종_세분화,업종_label
0,평균,18.681923,35.363958,16.939203,3.707248,1.01484,124.411859,-16.248642,-10.247494,65.018069,81.617527,1521.698501,0.739408,1.398059,4.958299e+09,8.741003e+09,77.586656,88.316067,95.901746,59.212927,79.599948,6.208198,NaN,0.010208,21.240447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.188981,-41.177780,587.025438,0.668026,-1.057112,5.616693,1.128886e+11,6.492112e+09,7.490148e+09,평균,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,389500,46.030000,-39.460000,2.260000,-5.230000,-9.21000,-168.760000,-20.800000,-11.800000,20.220000,394.440000,340.070000,0.440000,1.390000,-1.917031e+09,6.075800e+07,100.000000,50.000000,100.000000,16.666667,66.666667,0.000000,NaN,0.023564,14.593801,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,불안정,불안정,2018.0,110.399585,-72.542571,263.452137,0.047736,-5.930419,4.000000,9.218357e+09,-1.917403e+09,-1.088044e+09,에스비비테크,일반 목적용 기계 제조업,"베어링, 감속기",2022-10-17,12월,류재완,http://www.sbb.co.kr/,경기도,C29,9.0
2,296640,81.840000,215.020000,-6.230000,3.880000,1.43000,68.580000,1.900000,5.160000,40.950000,144.170000,2371.700000,0.750000,2.160000,1.770280e+08,1.148444e+09,66.666667,75.000000,100.000000,66.666667,100.000000,0.000000,NaN,0.006766,17.328700,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,안정,안정,2018.0,90.942256,-63.629994,396.408093,0.470922,-0.354859,6.000000,9.313410e+09,1.478880e+08,4.809880e+08,이노룰스,소프트웨어 개발 및 공급업,"InnoRules, InnoProduct",2022-10-07,12월,김길곤,http://innorules.com,서울특별시,J58,0.0
3,417970,23.530000,44.460000,9.650000,25.300000,19.34000,768.480000,12.970000,16.970000,47.940000,108.600000,2005.450000,1.490000,3.330000,7.210181e+09,5.469434e+09,100.000000,75.000000,100.000000,100.000000,100.000000,0.000000,NaN,0.000343,26.200702,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,안정,안정,2018.0,57.427845,-3.584996,595.858592,0.863325,0.549708,7.000000,5.559806e+10,9.218748e+09,9.433205e+09,모델솔루션,특수 목적용 기계 제조업,"하이엔드 프로토타입 제조, 다품종 소량생산 서비스",2022-10-07,12월,우병일,http://www.model-solution.com,서울특별시,C29,9.0
4,314140,38.750000,31.690000,-5.020000,8.280000,5.66000,146.000000,6.740000,9.850000,58.820000,70.000000,1558.220000,0.840000,1.400000,4.363446e+09,6.995499e+09,66.666667,100.000000,100.000000,66.666667,100.000000,0.000000,NaN,0.000461,21.578064,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,안정,안정,2018.0,77.101658,-20.206862,577.005958,0.815275,-0.072114,6.000000,6.476281e+10,5.851262e+09,6.382369e+09,알피바이오,의약품 제조업,연질캡슐 제형의 의약품 및 건강기능식품,2022-09-29,12월,"윤재훈, 김남기",http://rpskorea.com,서울특별시,C21,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,950110,78.370000,-3.080000,-3.400000,0.920000,3.84000,34.320000,23.900000,5.720000,17.630000,467.080000,117.880000,0.160000,0.700000,7.052749e+09,-8.972330e+10,66.666667,25.000000,66.666667,83.333333,66.666667,0.965841,경계태세 단계 기업,0.005466,15.903376,안전,비해당,비해당,비해당,비해당,비해당,비해당,비해당,관리종목아님,적정,안정,안정,안정,2021.0,110.399585,-72.542571,263.452137,0.047736,-5.930419,4.000000,2.951487e+10,1.011052e+10,1.688230e+09,SBI핀테크솔루션즈,소프트웨어 개발 및 공급업,전자결제대행 및 핀테크 비스,2012-12-17,03월,카네코 유이치,http://https://www.sbi-finsol.co.jp/kr/,일본,J58,0.0
5113,46970,2.980000,99.940000,-0.770000,5.310000,22.63000,88.700000,13.820000,3.240000,64.990000,53.870000,154.510000,1.640000,3.310000,1.335246e+10,-2.713149e+09,66.666667,75.000000,100.000000,83.333333,66.666667,3.323928,안전 단계 기업,0.000051,15.822916,안전,비해당,비해당,비해당,해당,비해당,해당,해당,관리종목,적정,안정,안정,안정,2021.0,4.120000,1.520000,464.000000,0.000000,-2.506494,6.

In [23]:
df['부실기업판별_Z_score'].isna().sum()

377

In [24]:
df.to_excel('명성은.xlsx')